In [1]:
import pandas as pd
import requests #For downloading website
from bs4 import BeautifulSoup # parsing the website
import time # Put system to sleep
import random
import country_converter as coco
import re
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from selenium import webdriver

In [2]:
url = "https://rajyasabha.nic.in/rsnew/member_site/newsessionwise_attendance.aspx" #Main page url
page=requests.get(url) #getting the html page
page.status_code #Check for connection

200

In [3]:
soup = BeautifulSoup(page.content, "html.parser") #creating 'soup' by parsing the html page

In [4]:
BIT_content = [i.get_text() for i in soup.select('div>div')]
BIT_content[0:10]

#ctl00_ContentPlaceHolder1_DataGrid1 > tbody > tr:nth-child(18) > td:nth-child(2)

['skip to main content | screen reader access',
 "\n\nHome\n \n\n\n\n  Rajya Sabha\n\nCouncil of States\nChairman »\n\nPresent\nFormer\n\n\nDeputy Chairman »\n\nPresent\nFormer\n\n\nLeader of the House »\n\nPresent\nFormer\n\n\nLeader of Opposition »\n\nPresent\nFormer\n\n\nPanel of Vice-Chairmen\nSecretary-General »\n\nPresent\nFormer\n\n\nImportant Parliamentary Terms\n\n\n\n  Today in Rajya Sabha\n\nToday at a Glance \nList of Business »\n\nCurrent day\nPrevious day\nNext day\n\n\nPapers  to be laid on the Table »\n\nCurrent\nPrevious day\n\n\nBulletin Part-I »\n\nLatest\n\n\nBulletin Part-II »\n\nLatest\n\n\nSynopsis »\n\nLatest\n\n\nUncorrected Debates »\n\n\nLatest\n\n\nMember's Birthday »\n\nToday\nPrevious day\nNext day\nMonth-wise\n\n\nCommittee Meetings »\n\nToday\nThis week\nNext week\nThis Month\n\n\nQuestions List »\n\nStarred »\n\nAnswers given/tabled\n\n\nUnstarred »\n\nAnswers tabled\n\n\n\n\n\n\n\n  Business\n\nProvisional Calendar\nList of Business\nPapers  to be laid

In [22]:
driver = webdriver.Chrome()
driver.get('https://rajyasabha.nic.in/rsnew/member_site/newsessionwise_attendance.aspx')
time.sleep(15)

#Step1 - find no of rows
row_elements = driver.find_elements_by_xpath("//*[@id='ctl00_ContentPlaceHolder1_DataGrid1']/tbody/tr/td]")

ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [5]:
idAktWert = 'ctl00_ContentPlaceHolder1_DataGrid1'
aktWert = soup.select("#" + idAktWert)[0]    #Note: I have used Index to select the first element in list. 
print(aktWert.text)




SESSION NOSESSIONFROMTO


255
Winter Session29-11-202123-12-2021


254
Monsoon Session19-07-202111-08-2021


253
Budget Session29-01-202125-03-2021


252
Monsoon Session14-09-202024-09-2020


251
Budget Session31-01-202003-04-2020


250
Winter Session18-11-201913-12-2019


249
Monsoon Session20-06-201907-08-2019


248
Budget Session31-01-201913-02-2019


247
Winter Session11-12-201809-01-2019


246
Monsoon Session18-07-201810-08-2018


245part2
Budget Session05-03-201806-04-2018


245part1
Budget Session29-01-201809-02-2018


244
Winter Session15-12-201705-01-2018


243
Monsoon Session17-07-201711-08-2017


242part2
Budget Session09-03-201712-04-2017


242part1
Budget Session31-01-201709-02-2017


241
Winter Session16-11-201616-12-2016


240
Monsoon Session18-07-201612-08-2016


239
Budget Session25-04-201613-05-2016


238
Budget Session23-02-201616-03-2016


237
Winter Session26-11-201523-12-2015


236
Monsoon Session21-07-201513-08-2015


235
Budget Session23-04-201513-05-2015


23

In [6]:
sessions = list(aktWert.text.split())
sessions[0:10]

['SESSION',
 'NOSESSIONFROMTO',
 '255',
 'Winter',
 'Session29-11-202123-12-2021',
 '254',
 'Monsoon',
 'Session19-07-202111-08-2021',
 '253',
 'Budget']

In [7]:
session_no = sessions [2::3] 
session_name = sessions [3::3] 
session_dates = sessions [4::3]
session_end_date = [i[-10:] for i in session_dates]
session_start_date = [i[-20:-10] for i in session_dates]

In [8]:
Sessions_meta_data = pd.DataFrame({'session_no':session_no, 'session_name':session_name,\
              'session_end_date':session_end_date, 'session_start_date':session_start_date})
Sessions_meta_data.to_csv('Sessions_meta_data.csv')
Sessions_meta_data

,session_no,session_name,session_end_date,session_start_date
0,255,Winter,23-12-2021,29-11-2021
1,254,Monsoon,11-08-2021,19-07-2021
2,253,Budget,25-03-2021,29-01-2021
3,252,Monsoon,24-09-2020,14-09-2020
4,251,Budget,03-04-2020,31-01-2020
5,250,Winter,13-12-2019,18-11-2019
6,249,Monsoon,07-08-2019,20-06-2019
7,248,Budget,13-02-2019,31-01-2019
8,247,Winter,09-01-2019,11-12-2018
9,246,Monsoon,10-08-2018,18-07-2018


In [9]:
links = set() #initialize a set
for tag in soup.find_all('a'): #for each link found on the main page
    href = tag.attrs.get("href") #extract the url 
    links.update([href]) #Add it to the initialized links set

links = list(links)
links

['../../rsnew/rssorgchart/js.aspx',
 '',
 'http://164.100.47.5/member_attence/attend213_II.htm',
 'sessionwiseresults.aspx?vsessionno=225Part2',
 '../../rsnew/rss/Sanctioned_Strength.asp',
 '../../rsnew/member_site/minister.aspx',
 '../../rsnew/Calander/new_clock.asp',
 '../../rsnew/Committee_site/meetlist_thisweak.aspx',
 '../../rsnew/Questions/qcalender.aspx',
 'http://164.100.47.5/pic/frontpage.aspx',
 'sessionwiseresults.aspx?vsessionno=216',
 '../../rsnew/question/rsgroup.asp',
 '../../rsnew/member_site/advsearch.aspx',
 'http://164.100.47.5/member_attence/Members%20Attendance%20202.htm',
 'http://164.100.47.5/member_attence/Members_Attendance%20170Part-I.htm',
 '../../faq/qlist.aspx',
 "javascript:openurl('http://mpa.gov.in/');",
 'http://164.100.47.5/member_attence/Members_Attendance163.htm',
 '../../rsnew/Questions/qchart.aspx',
 '../../rsnew/member_site/EngHindimemberlist.aspx',
 '../../rsnew/member_site/mp_income_display.aspx',
 'http://164.100.47.5/member_attence/Members%20A

In [10]:
links_list_1 = [i for i in links if "sessionwiseresults.aspx?vsessionno" in i] # https://rajyasabha.nic.in/rsnew/member_site/

#'http://164.100.47.4/Members_Attendance/sessionwiseresults.aspx?vsessionno=212'


attendance_links = set() #initialize a set
for l in links_list_1: #for each link found on the main page

    if "http:" not in l: #format the url
        attendance_links.update(["https://rajyasabha.nic.in/rsnew/member_site/" + l]) #Add it to the initialized links set

attendance_links = list(attendance_links)
attendance_links

['https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=245part1',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=230',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=220',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=241',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=233',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=248',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=254',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=216',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=244',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=221',
 'https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=217',
 'https://rajyasabha.nic.in/rsnew/memb

In [48]:
links_list = [i for i in links if "Attendance" in i]

In [49]:
links_list

['http://164.100.47.5/member_attence/Members%20Attendance%20204.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20189-I.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20202.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20175.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20178.htm',
 'http://164.100.47.5/member_attence/Members_Attendance163.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20177.htm',
 'http://164.100.47.5/member_attence/Members_Attendance166.htm',
 'http://164.100.47.5/member_attence/Members_Attendance172.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20196.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20176.htm',
 'http://164.100.47.5/member_attence/Members_%20Attendance156.htm',
 'http://164.100.47.5/member_attence/Members%20Attendance%20192%20Part-I.htm',
 'http://164.100.47.5/member_attence/Members_%20Attendance165.htm',
 'http://164.100.47.5/member

In [92]:
data = pd.read_html("http://164.100.47.5/member_attence/Members%20Attendance%20204.htm")
df = pd.concat(data)
df.columns = df.iloc[0]
df

,DIV NO,NAME OF THE MEMBER,No. of days attended
0,DIV NO,NAME OF THE MEMBER,No. of days attended
1,1,"Singh, Dr. Manmohan (Assam)",PM
2,2,"Azad, Shri Ghulam Nabi (Jammu & Kashmir)",M
3,3,"Soni, Shrimati Ambika (Punjab)",36
4,4,"Pachouri, Shri Suresh (Madhya Pradesh)",M
...,...,...,...
246,246,"Singh, Smt. Maya",24
247,247,"Pilania, Dr. Gyan Prakash (Rajasthan)",17
248,248,"Lath, Shri Surendra",21
249,249,"Bhardwaj, Shri Suresh",19


In [89]:
session = 'http://164.100.47.5/member_attence/Members%20Attendance%20204.htm'
session = re.split('%',session)
session = session[-1]
session = session.split(".")[0]
session = session[-3:]
session

'204'

In [95]:
attendance = {}
for link in links_list: #for each country link
    
    print(link)
    
    session = link
    session = re.split('%',session)
    session = session[-1]
    session = session.split(".")[0]
    session = session[-3:]
    
    data = pd.read_html(link)
    data = pd.concat(data)
    data.columns = data.iloc[0]
    data = data.iloc[1: , :]
    
    data['session_no'] = session
    
    attendance['session_{}'.format(session)] = data
    
    time.sleep(random.uniform(.5,3))

http://164.100.47.5/member_attence/Members%20Attendance%20204.htm
http://164.100.47.5/member_attence/Members%20Attendance%20189-I.htm
http://164.100.47.5/member_attence/Members%20Attendance%20202.htm
http://164.100.47.5/member_attence/Members%20Attendance%20175.htm
http://164.100.47.5/member_attence/Members%20Attendance%20178.htm
http://164.100.47.5/member_attence/Members_Attendance163.htm
http://164.100.47.5/member_attence/Members%20Attendance%20177.htm
http://164.100.47.5/member_attence/Members_Attendance166.htm
http://164.100.47.5/member_attence/Members_Attendance172.htm
http://164.100.47.5/member_attence/Members%20Attendance%20196.htm
http://164.100.47.5/member_attence/Members%20Attendance%20176.htm
http://164.100.47.5/member_attence/Members_%20Attendance156.htm
http://164.100.47.5/member_attence/Members%20Attendance%20192%20Part-I.htm
http://164.100.47.5/member_attence/Members_%20Attendance165.htm
http://164.100.47.5/member_attence/Members%20Attendance198.htm
http://164.100.47.5/m

In [98]:
attendance['session_9-I']

,DIV NO,NAME OF THE MEMBER,No. of days attended,session_no
1,1,"Singh, Shri Jaswant",M,9-I
2,2,NaN,NaN,9-I
3,3,Shri Dilip Ray,M,9-I
4,4,Shri Bangaru Laxman,M,9-I
5,5,Shri Arun Shourie,M,9-I
...,...,...,...,...
246,246,NaN,NaN,9-I
247,247,Shri C. Apok Jamir,14,9-I
248,248,Shri Surednder Kumar Singh,16,9-I
249,249,Shri Maurice Kujur,12,9-I


In [101]:
connection = sqlite3.connect("MP_attendance.sqlite")
cursor = connection.cursor()

In [103]:
for key, value in attendance.items():
    data = attendance[key]
    data.to_sql(key, connection, index= False )
    
connection.commit()

# PART2 ATTENDANCE SCRAPING

In [11]:
### attendance_links -> use Selenium to go to each link and 
attendance_page = requests.get('https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=221') # Getting the html page using the respective country url
attendance_soup = BeautifulSoup(attendance_page.content,"html.parser") #getting the page content through html parser
attendance_soup  


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html xmlns="http://www.w3.org/1999/xhtml">
<head id="ctl00_Head1"><meta content="Rajya Sabha Members" name="description"/><meta content="Rajya,Sabha,Parliament,Members,Member" name="keywords"/><meta content="IE=EmulateIE8" http-equiv="X-UA-Compatible"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><title>
	Rajya Sabha : Member's Attendance
</title><link href="../css/lowContrast.css" id="ctl00_stylesheet" rel="stylesheet"/>
<script src="../js/displaynew.js" type="text/javascript"></script>
<script src="../js/doctextsizer.js" type="text/javascript"></script>
<script language="javascript">

     function HandleBackFunctionality() {
         if (window.event) {
             if (window.event.clientX < 40 && window.event.clientY < 0) {
                 alert("Browser back button is clicked...");
             }
             else {
                 

In [12]:
    #ctl00_ContentPlaceHolder1_GridView1 > tbody > tr:nth-child(6) > td:nth-child(3)
    
place_holder = 'ctl00_ContentPlaceHolder1_GridView1'
place_holder = attendance_soup.select("#" + place_holder)[0]    #Note: I have used Index to select the first element in list. 
a = print(place_holder.text)
a = pd.concat(a)
a



S.No.Division/ Seat No.Name of MemberStateNo of days member signed the register

10Shri M. Rajasekara MurthyKAR       11

21Dr. Manmohan SinghRJ        PM

33Smt. Ambika SoniPB        M

44Shri Prithviraj ChavanMH        M

55Shri  Anand SharmaHP        M

66Shri Murli DeoraMH        M

77Prof. Saif-ud-Din SozJ & K     23

88Smt. Mohsina KidwaiCHT       22

99Shri Rishang KeishingMN        21

1011Shri Janardan DwivediDL        23

1112Ms. Sushila TiriyaOR        16

1213Shri Rajeev ShuklaMH        21

1314Shri Narendra BudaniaRJ        22

1415*Dr. Prabha ThakurRJ        11

1516Smt. Viplove ThakurHP        20

1617Prof. Alka  Balram KshatriyaGJ        19

1718Shri Silvius CondpanAS        23

1819Shri Praveen RashtrapalGJ        21

1920Shri Parvez HashmiDL        20

2021Shri Shadi Lal BatraHR        22

2122Shri P. KannanPUD       6

2223Smt. Naznin FaruqueAS        18

2324Shri A.K. AntonyKR        M

2425Shri S.M. KrishnaKAR       M

2526Shri Ghulam Nabi AzadJ & K     M

2627Dr

TypeError: 'NoneType' object is not iterable

In [13]:
x= list(place_holder.text.split("   "))
trail_data =pd.DataFrame(x)

In [14]:
trail_data.head()

,0
0,\n\nS.No.Division/ Seat No.Name of MemberState...
1,
2,11\n\n21Dr. Manmohan SinghRJ
3,
4,PM\n\n33Smt. Ambika SoniPB


In [15]:
nan_value = float("NaN")
trail_data.replace("",nan_value, inplace=True)

In [17]:
trail_data.dropna(subset = [0], inplace=True)

In [18]:
trail_data.head()

,0
0,\n\nS.No.Division/ Seat No.Name of MemberState...
2,11\n\n21Dr. Manmohan SinghRJ
4,PM\n\n33Smt. Ambika SoniPB
6,M\n\n44Shri Prithviraj ChavanMH
8,M\n\n55Shri Anand SharmaHP


In [19]:
attendance_aspx = pd.read_excel('Attendance_aspx_format.xlsx')
attendance_aspx.head()

,S.No.,Division/ Seat No.,Name of Member,State,No of days member signed the register
0,1,0,Shri M. Rajasekara Murthy,KAR,11
1,2,1,Dr. Manmohan Singh,RJ,PM
2,3,3,Smt. Ambika Soni,PB,M
3,4,4,Shri Prithviraj Chavan,MH,M
4,5,5,Shri Anand Sharma,HP,M


In [20]:
list_of_states = list(attendance_aspx.State)

In [21]:
trail_data.columns = ['text']


In [22]:
trail_data.head()

,text
0,\n\nS.No.Division/ Seat No.Name of MemberState...
2,11\n\n21Dr. Manmohan SinghRJ
4,PM\n\n33Smt. Ambika SoniPB
6,M\n\n44Shri Prithviraj ChavanMH
8,M\n\n55Shri Anand SharmaHP


In [23]:
trail_data[['text_1', 'text_2']]= trail_data['text'].str.split('\n\n', n=1, expand=True)

In [ ]:
trail_data['text_2'][0] = trail_data['text'][0].split('\n\n')[-1]

In [24]:
trail_data.head()

,text,text_1,text_2
0,\n\nS.No.Division/ Seat No.Name of MemberState...,,S.No.Division/ Seat No.Name of MemberStateNo o...
2,11\n\n21Dr. Manmohan SinghRJ,11,21Dr. Manmohan SinghRJ
4,PM\n\n33Smt. Ambika SoniPB,PM,33Smt. Ambika SoniPB
6,M\n\n44Shri Prithviraj ChavanMH,M,44Shri Prithviraj ChavanMH
8,M\n\n55Shri Anand SharmaHP,M,55Shri Anand SharmaHP


In [25]:
trail_data['text_2'] = trail_data['text_2'].str.replace('\d+', '')
trail_data.head()

,text,text_1,text_2
0,\n\nS.No.Division/ Seat No.Name of MemberState...,,S.No.Division/ Seat No.Name of MemberStateNo o...
2,11\n\n21Dr. Manmohan SinghRJ,11,Dr. Manmohan SinghRJ
4,PM\n\n33Smt. Ambika SoniPB,PM,Smt. Ambika SoniPB
6,M\n\n44Shri Prithviraj ChavanMH,M,Shri Prithviraj ChavanMH
8,M\n\n55Shri Anand SharmaHP,M,Shri Anand SharmaHP


In [26]:
for state in list_of_states:
    print(state)
    
    trail_data['text_2'] = trail_data['text_2'].str.replace(state,"")

KAR
RJ
PB
MH
HP
MH
J & K
CHT
MN
DL
OR
MH
RJ
RJ
HP
GJ
AS
GJ
DL
HR
PUD
AS
KR
KAR
J & K
DL
MP
KR
PB
TN
GJ
KR
KAR
KAR
NOM.
TN
AP
AS
JHK
PB
UP
AP
MP
TN
AR
MH
OR
AP
TG
HR
AP
TN
HR
TG
NOM.
GOA
WB
AP
NOM.
J & K
AP
MP
KAR
HR
AP
JHK
MH
RJ
TG
AS
WB
AP
CHT
MH
WB
BR
TN
UP
WB
BR
TN
WB
WB
BR
BR
BR
MH
TN
OR
WB
WB
NOM.
NOM.
NOM.
NOM.
NOM.
NOM.
NOM.
KAR
BR
MH
TN
TN
NOM.
WB
NG
OR
SK
MGH
MH
MH
TN
TN
J & K
WB
UP
UP
WB
WB
OR
BR
UP
WB
UP
BR
UP
TN
WB
OR
BR
UP
UP
UP
TN
KR
UP
UP
UP
AP
MH
WB
AP
KR
UP
AP
UP
UP
UP
WB
AP
UP
KAR
UP
UP
UP
UP
WB
BR
TR
KR
KR
KR
UP
UP
UP
UP
UP
MH
RJ
HP
JHK
KAR
BR
MP
BR
MH
BR
TN
JHK
OR
UP
UP
AS
PB
TN
CHT
MP
GJ
MP
MH
PB
BR
CHT
RJ
MP
UP
OR
AS
BR
TN
MH
PB
MZ
GJ
MH
MP
MP
OR
OR
HR
AP
TG
TN
BR
HP
UP
KAR
MP
UTK
BR
RJ
MP
GJ
MH
RJ
PB
KAR
CHT
KAR
UTK
GJ
MP
GJ
RJ
JHK
GJ
GJ


In [27]:
trail_data['text_1'] = trail_data['text_1'].shift(-1)

In [28]:
trail_data.head()

,text,text_1,text_2
0,\n\nS.No.Division/ Seat No.Name of MemberState...,11,S.No.Division/ Seat No.Name of MemberStateNo o...
2,11\n\n21Dr. Manmohan SinghRJ,PM,Dr. Manmohan Singh
4,PM\n\n33Smt. Ambika SoniPB,M,Smt. Ambika Soni
6,M\n\n44Shri Prithviraj ChavanMH,M,Shri Prithviraj Chavan
8,M\n\n55Shri Anand SharmaHP,M,Shri Anand Sharma


In [30]:

trail_data = trail_data[['text_1', 'text_2']]
trail_data.columns = ['Days_attended', 'Member_name']
trail_data.head()

,Days_attended,Member_name
0,11,10Shri M. Rajasekara MurthyKAR
2,PM,Dr. Manmohan Singh
4,M,Smt. Ambika Soni
6,M,Shri Prithviraj Chavan
8,M,Shri Anand Sharma


In [89]:
session_nos = pd.DataFrame(attendance_links)
session_nos.columns = ['link']
session_nos[['url', 'session_no']] = session_nos.link.str.split('=', expand=True )
session_nos.head()

,link,url,session_no
0,https://rajyasabha.nic.in/rsnew/member_site/se...,https://rajyasabha.nic.in/rsnew/member_site/se...,246
1,https://rajyasabha.nic.in/rsnew/member_site/se...,https://rajyasabha.nic.in/rsnew/member_site/se...,219Part2
2,https://rajyasabha.nic.in/rsnew/member_site/se...,https://rajyasabha.nic.in/rsnew/member_site/se...,238
3,https://rajyasabha.nic.in/rsnew/member_site/se...,https://rajyasabha.nic.in/rsnew/member_site/se...,251
4,https://rajyasabha.nic.in/rsnew/member_site/se...,https://rajyasabha.nic.in/rsnew/member_site/se...,253


In [33]:
attendance_links[0].split('=')[-1]

'245part1'

# BUILDING THE SECOND SCRAPPER

In [35]:
attendance_2 = {}
for link in attendance_links: #for each country link
    
    print(link)
    
    session = link.split('=')[-1]
    
    attendance_page = requests.get(link) # Getting the html page using the respective country url
    attendance_soup = BeautifulSoup(attendance_page.content,"html.parser") #getting the page content through html parser

    place_holder = 'ctl00_ContentPlaceHolder1_GridView1'
    place_holder = attendance_soup.select("#" + place_holder)[0]    #Note: I have used Index to select the first element in list. 

    x= list(place_holder.text.split("   "))
    trail_data =pd.DataFrame(x)

    nan_value = float("NaN")
    trail_data.replace("",nan_value, inplace=True)
    trail_data.dropna(subset = [0], inplace=True)
    trail_data.columns = ['text']

    trail_data[['text_1', 'text_2']]= trail_data['text'].str.split('\n\n', n=1, expand=True)
    trail_data['text_2'][0] = trail_data['text'][0].split('\n\n')[-1]
    trail_data['text_2'] = trail_data['text_2'].str.replace('\d+', '')

    for state in list_of_states:    
        trail_data['text_2'] = trail_data['text_2'].str.replace(state,"")

    trail_data['text_1'] = trail_data['text_1'].shift(-1)
    trail_data = trail_data[['text_1', 'text_2']]
    trail_data.columns = ['Days_attended', 'Member_name']


    
    trail_data['session_no'] = session
    
    attendance_2['session_{}'.format(session)] = trail_data
    
    time.sleep(random.uniform(.5,3))

https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=245part1
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=230
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=220
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=241
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=233
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=248
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=254
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=216
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=244
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=221
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=217
https://rajyasabha.nic.in/rsnew/member_site/sessionwiseresults.aspx?vsessionno=218

In [40]:
attendance_2['session_215']

,Days_attended,Member_name,session_no
0,7,Shri Ghulam Nabi Azad,215
1,3,Dr. Farooq Abdullah,215
2,6,Shri Mohammad Shafi,215
3,M,Prof. Saif-ud-Din Soz,215
4,PM,Dr. Manmohan Singh,215
...,...,...,...
462,8,*Dr. Gyan Prakash Pilania,215
464,7,Shri Jai Prakash Narayan Singh,215
466,3,Shri Natuji Halaji Thakor,215
468,3,Shri Bharatsinh Prabhatsinh Parmar,215


In [41]:
connection = sqlite3.connect("MP_attendance_scraper2.sqlite")
cursor = connection.cursor()

In [42]:
for key, value in attendance_2.items():
    data = attendance_2[key]
    data.to_sql(key, connection, index= False )

In [43]:
connection.commit()

In [44]:
connection.close()

In [49]:
attendance_files = pd.DataFrame(attendance_2.keys())
attendance_files.columns = ['session_file_name']
attendance_files.head()

,session_file_name
0,session_245part1
1,session_230
2,session_220
3,session_241
4,session_233


In [50]:
attendance_files.to_csv('attendance_file_names.csv')